## FSSPEC download for NWM RouteLink file for developing topologic relationships
This notebook demonstrates accessing the National Water Model (NWM) topological definition of the NWM channel routing simulation. The methods applied here utilize Zarr and FSSpec to retrieve the header for the file and then only the topology-definining fields: "link" and "to". Building the dataframe directly from these elements in the file from the web saves a 200Mb download and takes quite a bit less time than when obtaining the full file and operating from a local storage resource.

The key here is to note which operations take a long time: 
* The initial `SingleHdf5ToZarr` step is about 1 second
* The `.translate()` operation (inline in our example) is about 8 seconds
* Opening the dataset from the translated .json object is only a few milliseconds
* reading the "to" and "from" attributes into a pandas dataframe is 11 seconds

That last step would be a lot longer if all variables were downloaded. 

In [ ]:
import fsspec
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr

fs = fsspec.filesystem("http")

rl_nwm_url = "https://www.nco.ncep.noaa.gov/pmb/codes/nwprod/nwm.v2.2.0/parm/DOMAIN_WCOSS_Names/RouteLink_CONUS.nc"
with fs.open(rl_nwm_url) as f:
    %time    rl_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=0).translate()
    
    # Key example here: 
    # https://fsspec.github.io/kerchunk/test_example.html
 

The `kerchunk`-ing example that we started with had a number of other parameters... 
perhaps some might be reintroduced to make the data access even speedier!
e.g., ...
```py
fs = fsspec.filesystem('ftp', host="https://www.nco.ncep.noaa.gov/pmb")

with fs.open(rl_nwm_url, mode='rb', anon=True, default_fill_cache=False, default_cache_type='first') as f:
```
 ...
 
One thing that I specifically explored was the size of the `inline_threshold` setting. Smaller values definitely provided better results, though not a massivie improvement -- 9 seconds overall vs. 11 or so. 
```py
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url).translate() # 11.1 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=30000).translate() # 11.3 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=300).translate() # 11.2 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=10).translate() # 11.3 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=2).translate() # 9.8 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=1).translate() # 9.85 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=0).translate() # 9.83 s
    %time    rl_h5_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=-1).translate() # 9.54 s
```
Inlining the `.translate()` call vs. splitting seemed to be about equal, with inlining having the additional advantage of omitting the unused intermediate output. 
```py
    %time    rl_h5 = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=0)
    %time    rl_t = rl_h5.translate() # This translate MUST happen inside the context block
```
    

In [ ]:
backend_args = {
    "consolidated": False,
    "storage_options": {
        "fo": rl_t,
        # Adding these options returns a properly dimensioned but otherwise null dataframe
        # "remote_protocol": "https",
        # "remote_options": {'anon':True}
    },
}
%time ds = xr.open_dataset("reference://", engine="zarr", backend_kwargs=backend_args,)

In [ ]:
subslice = [
    "link",
    "to",
]
%time df = ds[subslice].to_dataframe().astype({"link": int, "to": int})

In [ ]:
df

## Create a topology
With the downloaded Route_Link, we can generate the topology of the CONUS river network

In [ ]:
df = df.set_index("link")
df

In [ ]:
# NOT Used yet...
def build_mask(param_df, mask_file_path, mask_key):
    data_mask = nhd_io.read_mask(
        mask_file_path,
    )

    return param_df.filter(data_mask.iloc[:, [mask_key]], axis=0)

In [ ]:
import nhd_network
from functools import partial


def replace_downstreams(data, downstream_col, terminal_code):
    ds0_mask = data[downstream_col] == terminal_code
    new_data = data.copy()
    new_data.loc[ds0_mask, downstream_col] = ds0_mask.index[ds0_mask]

    # Also set negative any nodes in downstream col not in data.index
    new_data.loc[~data[downstream_col].isin(data.index), downstream_col] *= -1
    return new_data


def organize_independent_networks(connections):
    rconn = nhd_network.reverse_network(connections)
    independent_networks = nhd_network.reachable_network(rconn)
    reaches_bytw = {}
    for tw, net in independent_networks.items():
        path_func = partial(nhd_network.split_at_junction, net)
        reaches_bytw[tw] = nhd_network.dfs_decomposition(net, path_func)

    return independent_networks, reaches_bytw, rconn

In [ ]:
df = df.sort_index()
df = replace_downstreams(df, "to", 0)

In [ ]:
connections = nhd_network.extract_connections(df, "to")

In [ ]:
independent_networks, reaches_bytw, rconn = organize_independent_networks(connections)

### So, what?
At this point we have a couple of objects representing the U.S. stream network (or another country or region, if you snuck a mask or different base route-link file in there!).
* `connections` is a dictionary of each `link` and the `to` downstream link id. All connections that point
to a null downstream (i.e., they flow off the map into the ocean or into an interior terminal basin) have
been massaged so that they point to an id which is the negative of the last valid segment id.
* `rconn` is the reverse dictionary of connections. Unlike the connections DAG which is strictly
coalescing, the `rconn` values contain multiple values where junctions split and the value list may
contain multiple upstream `link` ids for each of the incoming channels to a junction.
* `independent_networks` is a grouping of the rconn dictionary into connections that are related
topologically to a single tailwater. THIS IS NOT ORDERED (except by whatever falls out of the original reversal of the connections dictionary.)
* `reaches_bytw` is perhaps the most mysterious. It is a topologically ordered list of lists for
each tailwater that, if traversed in order, guarantees that each leaf of the tailwater DAG is touched
before any downstream junction is traversed.

### Example:
We can run simple script to find the networks of any given size. We choose a size of 8 so we can diagram things more easily.

In [ ]:
d = independent_networks
for k in sorted(d, key=lambda k: len(d[k]), reverse=True):
    if len(d[k]) == 8:
        print(k, len(d[k]))

From the list, we choose a random network that has only 8 channel segments, -20427622. 
If we examine the original dataframe, we can learn the lat-lon coordinates of our segment...

In [ ]:
segment = -20427622
print(segment, len(independent_networks[segment]))

print(
    df.loc[-segment]
)  # remember, we need the -segment because we've labeled the tailwaters with a dummy downstream terminal value

... and plot it's position on a map:



In [ ]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

_df = df.loc[[-segment]]
_geometry = [Point(xy) for xy in zip(_df["lon"], _df["lat"])]
_gdf = GeoDataFrame(_df, geometry=_geometry)

world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
_gdf.plot(ax=world.plot(figsize=(10, 6)), marker="o", color="red", markersize=15)

Or, we can use a slightly more sophisticated map to discover that we have chosen an interior basin near Phoenix. 

In [ ]:
import folium

# create a map
this_map = folium.Map(prefer_canvas=True)


def plotDot(point):
    """input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map"""
    folium.CircleMarker(location=[point.lat, point.lon], radius=8, weight=5).add_to(
        this_map
    )


# use df.apply(,axis=1) to "iterate" through every row in your dataframe
_df.apply(plotDot, axis=1)


# Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

# Save the map to an HTML file
this_map.save("simple_dot_plot.html")

this_map

So with that context for our tiny little drainage, we look at the connections and rconn dictionary results. 

In [ ]:
print(f"{connections[-segment]} the last item in the DAG points to our tailwater")
print(
    f"{rconn[segment]} ... and the upstream looking connection from the tailwater points to the last element of the DAG"
)

The `independent_networks` dictionary will show us which are the 8 segments in the DAG, each with their corresponding upstream neighbor or neighbors, which looks like the following for our example ...

```
independent_networks[segment]
{20427706: [20429532],
 20429540: [],
 20427622: [20427704, 20427706],
 20429612: [],
 20429616: [],
 20427704: [20429540],
 -20427622: [20427622],
 20429532: [20429612, 20429616]}
 ```

In [106]:
print(segment)
independent_networks[segment]

-20427622


{20427706: [20429532],
 20429540: [],
 20427622: [20427704, 20427706],
 20429612: [],
 20429616: [],
 20427704: [20429540],
 -20427622: [20427622],
 20429532: [20429612, 20429616]}

... and the `reaches_bytw` object gives the order of these in reaches between junctions using DFS ordering only reveresed, to start at the leaves.

for our example, this looks like
```
reaches_bytw[-20427622]

[[20429540, 20427704],
 [20429612],
 [20429616],
 [20429532, 20427706],
 [20427622, -20427622]]

```

In [105]:
print(segment)
reaches_bytw[segment]

-20427622


[[20429540, 20427704],
 [20429612],
 [20429616],
 [20429532, 20427706],
 [20427622, -20427622]]

You'll have to look at this for a minute, but trust me, you can derive the following topological
map from those two pieces of information. (Technically, you would only need the `independent_networks`
information, but it's nice to get confirmation from the other object.)
```
upstream...

             20429612     20429616
                 ├────────────┘
 20429540    20429532
    │            │
 20427704    20427706
    ├────────────┘
 20427622
    │
-20427622

downstream...
```

We can chain the segment IDs together -- this can be useful for instance if we want to query all IDs in a given basin. 

In [104]:
from itertools import chain

list(chain(*reaches_bytw[segment]))

[20429540,
 20427704,
 20429612,
 20429616,
 20429532,
 20427706,
 20427622,
 -20427622]